In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../data/clean_listings.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset shape: (396, 13)

Columns: ['location', 'property_type', 'bedrooms', 'bathrooms', 'parking', 'size_sqft', 'price_kes', 'price_per_sqft', 'listing_month', 'listing_year', 'location_tier', 'total_rooms', 'bed_bath_ratio']


,location,property_type,bedrooms,bathrooms,parking,size_sqft,price_kes,price_per_sqft,listing_month,listing_year,location_tier,total_rooms,bed_bath_ratio
0,Westlands,Apartment,1,1,1,732,8940000,12213.11,2,2026,High End,2,1.0
1,Westlands,Apartment,1,1,1,732,8940000,12213.11,2,2026,High End,2,1.0
2,Syokimau,Apartment,2,2,1,990,7500000,7575.76,2,2026,Mid Range,4,1.0
3,Westlands,Apartment,1,1,1,753,9200000,12217.80,2,2026,High End,2,1.0
4,Westlands,Apartment,1,1,1,721,9000000,12482.66,2,2026,High End,2,1.0


In [2]:
print("=" * 45)
print("QUESTION 1: Which locations are most expensive?")
print("=" * 45)
location_prices = (df.groupby('location')['price_kes']
                   .agg(['median', 'count'])
                   .sort_values('median', ascending=False))
location_prices['median_millions'] = (location_prices['median'] / 1e6).round(2)
print(location_prices.head(10))

print("\n" + "=" * 45)
print("QUESTION 2: How strongly does size affect price?")
print("=" * 45)
correlation = df['size_sqft'].corr(df['price_kes'])
print(f"Correlation between size and price: {correlation:.3f}")
print("(1.0 = perfect relationship, 0 = no relationship)")
if correlation > 0.7:
    print("Verdict: STRONG positive relationship")
elif correlation > 0.4:
    print("Verdict: MODERATE positive relationship")
else:
    print("Verdict: WEAK relationship — other factors matter more")

print("\n" + "=" * 45)
print("QUESTION 3: What drives price the most?")
print("=" * 45)
numeric_cols = ['bedrooms', 'bathrooms', 'size_sqft', 
                'parking', 'total_rooms']
correlations = df[numeric_cols].corrwith(df['price_kes']).sort_values(ascending=False)
print(correlations.round(3))

QUESTION 1: Which locations are most expensive?
                   median  count  median_millions
location                                         
Kiambu Rd      35000000.0      3            35.00
Runda          35000000.0      4            35.00
Lavington      30000000.0      2            30.00
Muthangari     27790000.0      1            27.79
Spring Valley  23100000.0      1            23.10
Riverside      18300000.0      8            18.30
Kilimani       14400000.0     71            14.40
Westlands      13000000.0    233            13.00
Kileleshwa      9210000.0     54             9.21
Syokimau        8000000.0     14             8.00

QUESTION 2: How strongly does size affect price?
Correlation between size and price: 0.782
(1.0 = perfect relationship, 0 = no relationship)
Verdict: STRONG positive relationship

QUESTION 3: What drives price the most?
total_rooms    0.909
bathrooms      0.905
bedrooms       0.886
size_sqft      0.782
parking        0.297
dtype: float64


In [3]:

df_model = df.copy()

le_location = LabelEncoder()
le_type = LabelEncoder()
le_tier = LabelEncoder()

df_model['location_encoded'] = le_location.fit_transform(df_model['location'])
df_model['property_type_encoded'] = le_type.fit_transform(df_model['property_type'])
df_model['tier_encoded'] = le_tier.fit_transform(df_model['location_tier'])

features = [
    'bedrooms', 
    'bathrooms', 
    'size_sqft', 
    'parking',
    'total_rooms',
    'location_encoded',
    'property_type_encoded',
    'tier_encoded',
    'listing_month'
]

target = 'price_kes'

X = df_model[features]
y = df_model[target]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {features}")

Features shape: (396, 9)
Target shape: (396,)

Feature columns: ['bedrooms', 'bathrooms', 'size_sqft', 'parking', 'total_rooms', 'location_encoded', 'property_type_encoded', 'tier_encoded', 'listing_month']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      
    random_state=42     
)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"\nTraining = {len(X_train)/len(X)*100:.0f}% of data")
print(f"Testing = {len(X_test)/len(X)*100:.0f}% of data")

Training samples: 316
Testing samples: 80

Training = 80% of data
Testing = 20% of data


In [5]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


y_pred = lr_model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("=" * 45)
print("LINEAR REGRESSION — BASELINE PERFORMANCE")
print("=" * 45)
print(f"\nMAE:  KES {mae:,.0f}")
print(f"RMSE: KES {rmse:,.0f}")
print(f"R²:   {r2:.3f}")

print(f" On average, predictions are off by KES {mae:,.0f}")
print(f" The model explains {r2*100:.1f}% of price variation")

LINEAR REGRESSION — BASELINE PERFORMANCE

MAE:  KES 2,285,002
RMSE: KES 3,741,978
R²:   0.806

In plain English:
→ On average, predictions are off by KES 2,285,002
→ The model explains 80.6% of price variation


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.scatter(y_test / 1e6, y_pred / 1e6, alpha=0.5, color='steelblue')
min_val = min(y_test.min(), y_pred.min()) / 1e6
max_val = max(y_test.max(), y_pred.max()) / 1e6
ax1.plot([min_val, max_val], [min_val, max_val], 
         'r--', linewidth=2, label='Perfect prediction')
ax1.set_xlabel('Actual Price (KES Millions)')
ax1.set_ylabel('Predicted Price (KES Millions)')
ax1.set_title('Actual vs Predicted Prices')
ax1.legend()

residuals = y_test - y_pred
ax2.scatter(y_pred / 1e6, residuals / 1e6, alpha=0.5, color='coral')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted Price (KES Millions)')
ax2.set_ylabel('Error (KES Millions)')
ax2.set_title('Prediction Errors (Residuals)')

plt.tight_layout()
plt.savefig('../docs/baseline_model.png', dpi=150, bbox_inches='tight')
plt.show()

print("Charts saved")

In [ ]:
results = {
    'model': ['Linear Regression (Baseline)'],
    'MAE_kes': [round(mae)],
    'RMSE_kes': [round(rmse)],
    'R2': [round(r2, 3)]
}

results_df = pd.DataFrame(results)
results_df.to_csv('../data/model_results.csv', index=False)

print("Model results saved.")
print(results_df)